# Eating in Buenos Aires on a budget

<a id='content' />

## Contents

1. [Web Scraping](#scraping)
2. [Data Cleansing](#cleansing)
3. [Mapping](#mapping)
4. [Bonus track: currency volatility](#volatility)

In [1]:
#Importing packages
import pandas as pd
import numpy as np
import re
import time

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

import geopy
import folium
from folium.plugins import MarkerCluster

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

<a id='scraping' />

## Web Scraping
[(back to top)](#content)

In [ ]:
#Load webdriver
driver = webdriver.Chrome('/Users/nicolamori/Library/Application Support/binman_chromedriver/mac64/85.0.4183.38/chromedriver')

#Open page
driver.get('https://www.tripadvisor.com.ar/Restaurants-g312741-Buenos_Aires_Capital_Federal_District.html')
time.sleep(5)

#Filter by cheap restaurants
driver.find_element_by_xpath('//*[@id="component_47"]/div/div[6]/div[2]/div[1]/div/label/div/span').click()
time.sleep(3)

In [ ]:
%%time

#Empty list
total = []

#Number of pages
npages = int(driver.find_element_by_xpath('//*[@id="EATERY_LIST_CONTENTS"]/div[2]/div/div/a[6]').text)

for h in range(npages):
    
    #Restaurants elements
    container = driver.find_elements_by_class_name('_15_ydu6b')
    num_page_items = len(container)
    
    for i in range(num_page_items):
        #Record original tab
        window_before = driver.window_handles[0]
        
        #Open tab
        element = container[i]
        driver.execute_script("arguments[0].click();", element)
        
        #Switch to new tab
        window_after = driver.window_handles[1]
        driver.switch_to.window(window_after)
        time.sleep(1)
        
        #Extract data with covid19 section
        try: 
            driver.find_element_by_xpath('//*[@data-tab="TABS_COVID19"]')
            
            name = driver.find_element_by_xpath('//*[@class="_3a1XQ88S"]').text
            try:cuisines = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[2]/div/div[2]/div/div/div[2]/div[2]/div[2]').text
            except NoSuchElementException: cuisines = None  
            try: rank = driver.find_element_by_xpath('//*[@class="_15QfMZ2L"]').text
            except NoSuchElementException: rank = None  
            nreviews = driver.find_element_by_xpath('//*[@class="_3Wub8auF"]').text
            
            address = driver.find_element_by_xpath('//*[@class="_2saB_OSe"]').text
            try:phone = driver.find_element_by_xpath('//*[@id="taplc_top_info_0"]/div/div/div[3]/span[2]/span/span[2]/a').text
            except NoSuchElementException: website = None  
            try: website = driver.find_element_by_xpath('//*[@id="taplc_top_info_0"]/div/div/div[3]/span[3]/span/a').get_attribute("href")
            except NoSuchElementException: website = None  
            
            try:score = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[2]/div/div[1]/div/div[1]/div[1]/span[2]/span').get_attribute("class")
            except NoSuchElementException: score = None  
            try:food = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[2]/div/div[1]/div/div[3]/div[2]/div[1]/span[3]/span').get_attribute("class")
            except NoSuchElementException: food = None
            try:service = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[2]/div/div[1]/div/div[3]/div[2]/div[2]/span[3]/span').get_attribute("class")
            except NoSuchElementException: service = None
            try:value = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[2]/div/div[1]/div/div[3]/div[2]/div[3]/span[3]/span').get_attribute("class")
            except NoSuchElementException: value = None
        
        #Extract data without covid19 section
        except NoSuchElementException:        
            
            name = driver.find_element_by_xpath('//*[@class="_3a1XQ88S"]').text
            try: cuisines = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[1]/div/div[2]/div/div/div[2]/div[2]/div[2]').text
            except NoSuchElementException: cuisines = None
            try: rank = driver.find_element_by_xpath('//*[@class="_15QfMZ2L"]').text
            except NoSuchElementException: rank = None  
            nreviews = driver.find_element_by_xpath('//*[@class="_3Wub8auF"]').text
            
            address = driver.find_element_by_xpath('//*[@class="_2saB_OSe"]').text
            try:phone = driver.find_element_by_xpath('//*[@id="taplc_top_info_0"]/div/div/div[3]/span[2]/span/span[2]/a').text
            except NoSuchElementException: website = None  
            try: website = driver.find_element_by_xpath('//*[@id="taplc_top_info_0"]/div/div/div[3]/span[3]/span/a').get_attribute("href")
            except NoSuchElementException: website = None  
            
            try:score = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[1]/div/div[1]/div/div[1]/div[1]/span[2]/span').get_attribute("class")
            except NoSuchElementException: score = None  
            try:food = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[1]/div/div[1]/div/div[3]/div[2]/div[1]/span[3]/span').get_attribute("class")
            except NoSuchElementException: food = None
            try:service = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[1]/div/div[1]/div/div[3]/div[2]/div[2]/span[3]/span').get_attribute("class")
            except NoSuchElementException: service = None
            try:value = driver.find_element_by_xpath('//*[@id="taplc_detail_overview_cards_0"]/div/div[1]/div/div[1]/div/div[3]/div[2]/div[3]/span[3]/span').get_attribute("class")
            except NoSuchElementException: value = None
        
        #Add data to list
        new = ((name, cuisines, rank, nreviews, address, phone, website, score, food, service, value))
        total.append(new)
        
        #Close tab and switch to main window
        driver.close()
        driver.switch_to.window(window_before)
    
    #Switch to next page
    try:
        if h==0: element2 = driver.find_element_by_xpath('//*[@id="EATERY_LIST_CONTENTS"]/div[2]/div/a')
        else: element2 = driver.find_element_by_xpath('//*[@id="EATERY_LIST_CONTENTS"]/div[2]/div/a[2]')        
        
        driver.execute_script("arguments[0].click();", element2)
        time.sleep(3)
    except NoSuchElementException: driver.close()

#List to dataframe
df = pd.DataFrame(total,columns=['name', 'cuisines', 'rank', 'nreviews', 'address', 'phone', 'website', 'score', 'food', 'service', 'value'])

df.to_csv('affordable_restaurants.tsv', sep='\t', index=False)

<a id='cleansing' />

## Data Cleansing
[(back to top)](#content)

In [2]:
restaurants=pd.read_csv('affordable_restaurants.tsv', sep='\t', index_col=0)
restaurants = restaurants.astype(str)

restaurants.head(1)

,name,cuisines,rank,nreviews,address,phone,website,score,food,service,value
0,La Choripanería,"Argentina, Latina, Comida rápida, Parrilla",#16 de 5.006 restaurantes en Buenos Aires,316 opiniones,"Bolívar 954 Local 42, Mercado de San Telmo, Bu...",+54 11 4244-9478,https://www.facebook.com/La-Choripaner%C3%ADa-...,ui_bubble_rating bubble_45,ui_bubble_rating bubble_50,ui_bubble_rating bubble_45,ui_bubble_rating bubble_45


In [3]:
#Scoring columns
temp=['score', 'food', 'service', 'value']

for i in temp:
    restaurants[i] = [j.replace('ui_bubble_rating bubble_','') for j in restaurants[i]]
    restaurants[i] = pd.to_numeric(restaurants[i], errors='coerce')/10

In [4]:
#Number of reviews
restaurants['nreviews'] = [i.replace(' opiniones','') for i in restaurants['nreviews']]
restaurants['nreviews'] = pd.to_numeric(restaurants['nreviews'], errors='coerce')

In [5]:
#Addresses format and removing not normalized rows
new_address=[]

for values in restaurants['address']:
    try:
        street=re.search("(.*?)\d+", values).group()+', CABA, Argentina'
    except AttributeError:
        street='Not normalized'
    
    new_address.append(street)

restaurants['new_address']=new_address

restaurants['new_address']=([i
                             .replace('Calle ','')
                             .replace('Avenida ','')
                             .replace('Av. ','')
                             for i in restaurants['new_address']])

restaurants = restaurants[restaurants['new_address'] != 'Not normalized']

In [6]:
#Rank format
restaurants['rank'] = [i.replace('restaurantes en Buenos Aires','') for i in restaurants['rank']]

restaurants.head(1)

,name,cuisines,rank,nreviews,address,phone,website,score,food,service,value,new_address
0,La Choripanería,"Argentina, Latina, Comida rápida, Parrilla",#16 de 5.006,316.0,"Bolívar 954 Local 42, Mercado de San Telmo, Bu...",+54 11 4244-9478,https://www.facebook.com/La-Choripaner%C3%ADa-...,4.5,5.0,4.5,4.5,"Bolívar 954, CABA, Argentina"


<a id='mapping' />

## Mapping
[(back to top)](#content)

In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="restaurants_coord")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

from tqdm import tqdm
tqdm.pandas()

restaurants['location'] = restaurants['new_address'].progress_apply(geocode)

restaurants['point'] = restaurants['location'].apply(lambda loc: tuple(loc.point) if loc else None)

/Users/nicolamori/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 829/829 [13:55<00:00,  1.01s/it]


In [15]:
restaurants = restaurants[restaurants.point.notnull()]


In [20]:
restaurants.to_csv('affordable_restaurants2.tsv', sep='\t', index=False)

In [ ]:
restaurants[['latitude', 'longitude', 'altitude']] = restaurants.point.str.split(",", expand=True)

restaurants['latitude'] = [i.replace('(','') for i in restaurants['latitude']]
restaurants.drop(['point','altitude'], axis=1, inplace=True)


In [ ]:
import unidecode


In [ ]:
restaurants['latitude']=pd.to_numeric(restaurants['latitude'], errors='coerce')
restaurants['longitude']=pd.to_numeric(restaurants['longitude'], errors='coerce')


In [ ]:
%%time

m = folium.Map(location=[-34.6083,-58.4212], tiles='cartodbpositron', zoom_start=12)



marker_cluster = MarkerCluster(name='cheap restaurants', overlay=True, control=False, icon_create_function=None)

for i in range(len(restaurants['latitude'])):
    location = restaurants['latitude'][i], restaurants['longitude'][i]
    marker = folium.Marker(location=location)
    popup = ('<b>Name:</b> {} <br><b>Cuisines:</b> {} <br><b>Rank:</b> {} <br><b>Reviews:</b> {} <br><b>Score:</b> {}  <br><b>Address:</b> {} <br><b>Phone:</b> {} <br><b>Website:</b> <a href={}>abc</a>'
             .format(re.sub(r'[^\w]', ' ', unidecode.unidecode(restaurants['name'][i])), 
                     restaurants['cuisines'][i],
                     restaurants['rank'][i],
                     restaurants['nreviews'][i],
                     restaurants['score'][i], 
                     restaurants['new_address'][i], 
                     restaurants['phone'][i], 
                     restaurants['website'][i]))
    folium.Popup(popup, max_width=500).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
#https://python-visualization.github.io/folium/plugins.html#folium-plugins
#https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/MarkerCluster.ipynb

# PENDIENTES
## corregir direcciones
## probar capas en el mapa